# Import necessari:

In [1]:
from flickrapi import FlickrAPI
import json
from PIL import Image
from io import StringIO
import urllib
from torch.utils.data.dataset import Dataset
from PIL import Image
from os import path
import numpy as np
from pathlib import Path
from pprint import pprint
from torch.utils.data import DataLoader
import os
import random
from torchvision import transforms
from skimage.transform import rescale, resize
from skimage import io
import torch
from torchvision import models
import torch.nn as nn
from torch.optim import SGD
from torchnet.meter import AverageValueMeter
from torchnet.logger import VisdomPlotLogger, VisdomSaver
from torch.autograd import Variable
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Dichiarazione costanti.

In [2]:
width = 256
height = 256
#Media e deviazione standard precalcolata per Squeezenet.
mean_squeezenet = [0.485, 0.456, 0.406]
std_squeezenet = [0.229, 0.224, 0.225]

# Dichiarazione di classi e funzioni.

In [3]:
class ScenesDataset(Dataset):
    def __init__(self,base_path,txt_list,transform=None):
        #conserviamo il path alla cartella contenente le immagini
        self.base_path=base_path
        #carichiamo la lista dei file
        #sarà una matrice con n righe (numero di immagini) e 2 colonne (path, etichetta)
        self.images = np.loadtxt(txt_list,dtype=str,delimiter=',')
        #conserviamo il riferimento alla trasformazione da applicare
        self.transform = transform
    def __getitem__(self, index):
        #recuperiamo il path dell'immagine di indice index e la relativa etichetta
        f,c = self.images[index]
        #carichiamo l'immagine utilizzando PIL
        im = Image.open(path.join(self.base_path, f)).convert('RGB')
        #Resize:
        im = im.resize((width,height))
        #se la trasfromazione è definita, applichiamola all'immagine
        if self.transform is not None:
            im = self.transform(im)
        #convertiamo l'etichetta in un intero
        label = int(c)
        #restituiamo un dizionario contenente immagine etichetta
        return {'image' : im, 'label':label}
    #restituisce il numero di campioni: la lunghezza della lista "images"
    def __len__(self):
        return len(self.images)

- Dato un oggetto di tipo ScenesDataset restituisce la media e la deviazione standard calcolata su tutte le immagini.

In [4]:
def get_mean_devst(dataset):
    m = np.zeros(3)
    for sample in dataset:
        m+= np.array(sample['image'].sum(1).sum(1)) #accumuliamo la somma dei pixel canale per canale
    #dividiamo per il numero di immagini moltiplicato per il numero di pixel
    m=m/(len(dataset)*width*height)
    #procedura simile per calcolare la deviazione standard
    s = np.zeros(3)
    for sample in dataset:
        s+= np.array(((sample['image']-torch.Tensor(m).view(3,1,1))**2).sum(1).sum(1))
    s=np.sqrt(s/(len(dataset)*width*height))
    #print("Medie",m)
    #print("Dev.Std.",s)
    return m, s

In [5]:
def IsImg(name):
    if str.__contains__(name, ".jpg"):
        return True
    return False

# Download immagini da flickr tramite Api search.
- Inserire FLICKR_PUBLIC e FLICKR_SECRET personali.

In [7]:
FLICKR_PUBLIC = 'fdb1863f5162502cf08eafc41ddc43e5'
FLICKR_SECRET = '1bcbf94ddbf054d9'

flickr = FlickrAPI(FLICKR_PUBLIC, FLICKR_SECRET, format='parsed-json')
extras='url_m'

for pg in range(13,43):
    cats = flickr.photos.search(text='cat', page=pg, per_page=500, extras=extras, safe_search=1, content_type = 1, sort="interestingness-desc")
    photos_cats = cats['photos']
    if pg == 1:
        count = 0
    else:
        count = pg*500-501
    for i in cats['photos']['photo']:
        try:
            count += 1
            url = i['url_m']
            image = io.imread(url)
            #io.imshow(image)
            #io.show()
            io.imsave("ImmaginiMaiViste/cat/cat_" + str(count) + ".jpg", np.array(image))
            print("Cat number-->", count, "saved.")
        except:
            print("*******************")
            print("Errore...")
            print("*******************")
    
    dogs = flickr.photos.search(text='dog', page=pg, per_page=500, extras=extras, safe_search=1, content_type = 1, sort="interestingness-desc")
    photos_dogs = dogs['photos']
    if pg == 1:
        count = 0
    else:
        count = pg*500-501
    for i in dogs['photos']['photo']:
        try:
            count += 1
            url = i['url_m']
            image = io.imread(url)
            #io.imshow(image)
            #io.show()
            io.imsave("ImmaginiMaiViste/dog/dog_" + str(count) + ".jpg", np.array(image))
            print("Dog number-->", count, "saved.")
        except:
            print("*******************")
            print("Errore...")
            print("*******************")
print("Finished.")

Cat number--> 6000 saved.
Cat number--> 6001 saved.
Cat number--> 6002 saved.
Cat number--> 6003 saved.
Cat number--> 6004 saved.
Cat number--> 6005 saved.
Cat number--> 6006 saved.
Cat number--> 6007 saved.
Cat number--> 6008 saved.
Cat number--> 6009 saved.
Cat number--> 6010 saved.
Cat number--> 6011 saved.
Cat number--> 6012 saved.
Cat number--> 6013 saved.
Cat number--> 6014 saved.
Cat number--> 6015 saved.
Cat number--> 6016 saved.
Cat number--> 6017 saved.
Cat number--> 6018 saved.
Cat number--> 6019 saved.
Cat number--> 6020 saved.
Cat number--> 6021 saved.
Cat number--> 6022 saved.
Cat number--> 6023 saved.
Cat number--> 6024 saved.
Cat number--> 6025 saved.
Cat number--> 6026 saved.
Cat number--> 6027 saved.
Cat number--> 6028 saved.
Cat number--> 6029 saved.
Cat number--> 6030 saved.
Cat number--> 6031 saved.
Cat number--> 6032 saved.
Cat number--> 6033 saved.
Cat number--> 6034 saved.
Cat number--> 6035 saved.
Cat number--> 6036 saved.
Cat number--> 6037 saved.
Cat number--

Cat number--> 6315 saved.
Cat number--> 6316 saved.
Cat number--> 6317 saved.
Cat number--> 6318 saved.
Cat number--> 6319 saved.
Cat number--> 6320 saved.
Cat number--> 6321 saved.
Cat number--> 6322 saved.
Cat number--> 6323 saved.
Cat number--> 6324 saved.
Cat number--> 6325 saved.
Cat number--> 6326 saved.
Cat number--> 6327 saved.
Cat number--> 6328 saved.
Cat number--> 6329 saved.
Cat number--> 6330 saved.
Cat number--> 6331 saved.
Cat number--> 6332 saved.
Cat number--> 6333 saved.
Cat number--> 6334 saved.
Cat number--> 6335 saved.
Cat number--> 6336 saved.
Cat number--> 6337 saved.
Cat number--> 6338 saved.
Cat number--> 6339 saved.
Cat number--> 6340 saved.
Cat number--> 6341 saved.
Cat number--> 6342 saved.
Cat number--> 6343 saved.
Cat number--> 6344 saved.
Cat number--> 6345 saved.
Cat number--> 6346 saved.
Cat number--> 6347 saved.
Cat number--> 6348 saved.
Cat number--> 6349 saved.
Cat number--> 6350 saved.
Cat number--> 6351 saved.
Cat number--> 6352 saved.
Cat number--

Dog number--> 6131 saved.
Dog number--> 6132 saved.
Dog number--> 6133 saved.
Dog number--> 6134 saved.
Dog number--> 6135 saved.
Dog number--> 6136 saved.
Dog number--> 6137 saved.
Dog number--> 6138 saved.
Dog number--> 6139 saved.
Dog number--> 6140 saved.
Dog number--> 6141 saved.
Dog number--> 6142 saved.
Dog number--> 6143 saved.
Dog number--> 6144 saved.
Dog number--> 6145 saved.
Dog number--> 6146 saved.
Dog number--> 6147 saved.
Dog number--> 6148 saved.
Dog number--> 6149 saved.
Dog number--> 6150 saved.
Dog number--> 6151 saved.
Dog number--> 6152 saved.
Dog number--> 6153 saved.
Dog number--> 6154 saved.
Dog number--> 6155 saved.
Dog number--> 6156 saved.
Dog number--> 6157 saved.
Dog number--> 6158 saved.
Dog number--> 6159 saved.
Dog number--> 6160 saved.
Dog number--> 6161 saved.
Dog number--> 6162 saved.
Dog number--> 6163 saved.
Dog number--> 6164 saved.
Dog number--> 6165 saved.
Dog number--> 6166 saved.
Dog number--> 6167 saved.
Dog number--> 6168 saved.
Dog number--

Dog number--> 6446 saved.
Dog number--> 6447 saved.
Dog number--> 6448 saved.
Dog number--> 6449 saved.
Dog number--> 6450 saved.
Dog number--> 6451 saved.
Dog number--> 6452 saved.
Dog number--> 6453 saved.
Dog number--> 6454 saved.
Dog number--> 6455 saved.
Dog number--> 6456 saved.
Dog number--> 6457 saved.
Dog number--> 6458 saved.
Dog number--> 6459 saved.
Dog number--> 6460 saved.
Dog number--> 6461 saved.
Dog number--> 6462 saved.
Dog number--> 6463 saved.
Dog number--> 6464 saved.
Dog number--> 6465 saved.
Dog number--> 6466 saved.
Dog number--> 6467 saved.
Dog number--> 6468 saved.
Dog number--> 6469 saved.
Dog number--> 6470 saved.
Dog number--> 6471 saved.
Dog number--> 6472 saved.
Dog number--> 6473 saved.
Dog number--> 6474 saved.
Dog number--> 6475 saved.
Dog number--> 6476 saved.
Dog number--> 6477 saved.
Dog number--> 6478 saved.
Dog number--> 6479 saved.
Dog number--> 6480 saved.
Dog number--> 6481 saved.
Dog number--> 6482 saved.
Dog number--> 6483 saved.
Dog number--

Cat number--> 6763 saved.
Cat number--> 6764 saved.
Cat number--> 6765 saved.
Cat number--> 6766 saved.
Cat number--> 6767 saved.
Cat number--> 6768 saved.
Cat number--> 6769 saved.
Cat number--> 6770 saved.
Cat number--> 6771 saved.
Cat number--> 6772 saved.
Cat number--> 6773 saved.
Cat number--> 6774 saved.
Cat number--> 6775 saved.
Cat number--> 6776 saved.
Cat number--> 6777 saved.
Cat number--> 6778 saved.
Cat number--> 6779 saved.
Cat number--> 6780 saved.
Cat number--> 6781 saved.
Cat number--> 6782 saved.
Cat number--> 6783 saved.
Cat number--> 6784 saved.
Cat number--> 6785 saved.
Cat number--> 6786 saved.
Cat number--> 6787 saved.
Cat number--> 6788 saved.
Cat number--> 6789 saved.
Cat number--> 6790 saved.
Cat number--> 6791 saved.
Cat number--> 6792 saved.
Cat number--> 6793 saved.
Cat number--> 6794 saved.
Cat number--> 6795 saved.
Cat number--> 6796 saved.
Cat number--> 6797 saved.
Cat number--> 6798 saved.
Cat number--> 6799 saved.
Cat number--> 6800 saved.
Cat number--

...

Dog number--> 20068 saved.
Dog number--> 20069 saved.
Dog number--> 20070 saved.
Dog number--> 20071 saved.
Dog number--> 20072 saved.
Dog number--> 20073 saved.
Dog number--> 20074 saved.
Dog number--> 20075 saved.
Dog number--> 20076 saved.
Dog number--> 20077 saved.
Dog number--> 20078 saved.
Dog number--> 20079 saved.
Dog number--> 20080 saved.
Dog number--> 20081 saved.
Dog number--> 20082 saved.
Dog number--> 20083 saved.
Dog number--> 20084 saved.
Dog number--> 20085 saved.
Dog number--> 20086 saved.
Dog number--> 20087 saved.
Dog number--> 20088 saved.
Dog number--> 20089 saved.
Dog number--> 20090 saved.
Dog number--> 20091 saved.
Dog number--> 20092 saved.
Dog number--> 20093 saved.
Dog number--> 20094 saved.
Dog number--> 20095 saved.
Dog number--> 20096 saved.
Dog number--> 20097 saved.
Dog number--> 20098 saved.
Dog number--> 20099 saved.
Dog number--> 20100 saved.
Dog number--> 20101 saved.
Dog number--> 20102 saved.
Dog number--> 20103 saved.
Dog number--> 20104 saved.
D

Dog number--> 20371 saved.
Dog number--> 20372 saved.
Dog number--> 20373 saved.
Dog number--> 20374 saved.
Dog number--> 20375 saved.
Dog number--> 20376 saved.
Dog number--> 20377 saved.
Dog number--> 20378 saved.
Dog number--> 20379 saved.
Dog number--> 20380 saved.
Dog number--> 20381 saved.
Dog number--> 20382 saved.
Dog number--> 20383 saved.
Dog number--> 20384 saved.
Dog number--> 20385 saved.
Dog number--> 20386 saved.
Dog number--> 20387 saved.
Dog number--> 20388 saved.
Dog number--> 20389 saved.
Dog number--> 20390 saved.
Dog number--> 20391 saved.
Dog number--> 20392 saved.
Dog number--> 20393 saved.
Dog number--> 20394 saved.
Dog number--> 20395 saved.
Dog number--> 20396 saved.
Dog number--> 20397 saved.
Dog number--> 20398 saved.
Dog number--> 20399 saved.
Dog number--> 20400 saved.
Dog number--> 20401 saved.
Dog number--> 20402 saved.
Dog number--> 20403 saved.
Dog number--> 20404 saved.
Dog number--> 20405 saved.
Dog number--> 20406 saved.
Dog number--> 20407 saved.
D

Cat number--> 20676 saved.
Cat number--> 20677 saved.
Cat number--> 20678 saved.
Cat number--> 20679 saved.
Cat number--> 20680 saved.
Cat number--> 20681 saved.
Cat number--> 20682 saved.
Cat number--> 20683 saved.
Cat number--> 20684 saved.
Cat number--> 20685 saved.
Cat number--> 20686 saved.
Cat number--> 20687 saved.
Cat number--> 20688 saved.
Cat number--> 20689 saved.
Cat number--> 20690 saved.
Cat number--> 20691 saved.
Cat number--> 20692 saved.
Cat number--> 20693 saved.
Cat number--> 20694 saved.
Cat number--> 20695 saved.
Cat number--> 20696 saved.
Cat number--> 20697 saved.
Cat number--> 20698 saved.
Cat number--> 20699 saved.
Cat number--> 20700 saved.
Cat number--> 20701 saved.
Cat number--> 20702 saved.
Cat number--> 20703 saved.
Cat number--> 20704 saved.
Cat number--> 20705 saved.
Cat number--> 20706 saved.
Cat number--> 20707 saved.
Cat number--> 20708 saved.
Cat number--> 20709 saved.
Cat number--> 20710 saved.
Cat number--> 20711 saved.
Cat number--> 20712 saved.
C

Cat number--> 20979 saved.
Cat number--> 20980 saved.
Cat number--> 20981 saved.
Cat number--> 20982 saved.
Cat number--> 20983 saved.
Cat number--> 20984 saved.
Cat number--> 20985 saved.
Cat number--> 20986 saved.
Cat number--> 20987 saved.
Cat number--> 20988 saved.
Cat number--> 20989 saved.
Cat number--> 20990 saved.
Cat number--> 20991 saved.
Cat number--> 20992 saved.
Cat number--> 20993 saved.
Cat number--> 20994 saved.
Cat number--> 20995 saved.
Cat number--> 20996 saved.
Cat number--> 20997 saved.
Cat number--> 20998 saved.
Dog number--> 20500 saved.
Dog number--> 20501 saved.
Dog number--> 20502 saved.
Dog number--> 20503 saved.
Dog number--> 20504 saved.
Dog number--> 20505 saved.
Dog number--> 20506 saved.
Dog number--> 20507 saved.
Dog number--> 20508 saved.
Dog number--> 20509 saved.
Dog number--> 20510 saved.
Dog number--> 20511 saved.
Dog number--> 20512 saved.
Dog number--> 20513 saved.
Dog number--> 20514 saved.
Dog number--> 20515 saved.
Dog number--> 20516 saved.
D

Dog number--> 20783 saved.
Dog number--> 20784 saved.
Dog number--> 20785 saved.
Dog number--> 20786 saved.
Dog number--> 20787 saved.
Dog number--> 20788 saved.
Dog number--> 20789 saved.
Dog number--> 20790 saved.
Dog number--> 20791 saved.
Dog number--> 20792 saved.
Dog number--> 20793 saved.
Dog number--> 20794 saved.
Dog number--> 20795 saved.
Dog number--> 20796 saved.
Dog number--> 20797 saved.
Dog number--> 20798 saved.
Dog number--> 20799 saved.
Dog number--> 20800 saved.
Dog number--> 20801 saved.
Dog number--> 20802 saved.
Dog number--> 20803 saved.
Dog number--> 20804 saved.
Dog number--> 20805 saved.
Dog number--> 20806 saved.
Dog number--> 20807 saved.
Dog number--> 20808 saved.
Dog number--> 20809 saved.
Dog number--> 20810 saved.
Dog number--> 20811 saved.
Dog number--> 20812 saved.
Dog number--> 20813 saved.
Dog number--> 20814 saved.
Dog number--> 20815 saved.
Dog number--> 20816 saved.
Dog number--> 20817 saved.
Dog number--> 20818 saved.
Dog number--> 20819 saved.
D

# Creo i file train e test txt con le relative classi.

In [8]:
train = open("train_immagini_mai_viste.txt","w+")
test = open("test_immagini_mai_viste.txt","w+")
random.seed(1234)
entries = os.listdir('ImmaginiMaiViste/cat/')
stop = 0
for entry in entries:
    #if( stop == 200):
        #break
    stop += 1
    if IsImg(entry):
        #if(random.randint(1, 100) <= 70):
        train.write("ImmaginiMaiViste/cat/" + entry + ", " + str(0) +"\n")
        #else:
            #test.write("ImmaginiMaiViste/cat/" + entry + ", " + str(0) + "\n")

entries = os.listdir('ImmaginiMaiViste/dog/')
stop = 0
for entry in entries:
    #if( stop == 200):
        #break
    stop += 1
    if IsImg(entry):        
        #if(random.randint(1, 100) <= 70):
        train.write("ImmaginiMaiViste/dog/" + entry + ", " + str(1) +"\n")
        #else:
        #test.write("ImmaginiMaiViste/dog/" + entry + ", " + str(1) +"\n")

train.close()
test.close()

# Creo l'oggetto dataset

In [ ]:
#Decommentare per ricalcolare se cambiano le immagini.
dataset_train = ScenesDataset('','train.txt',transform=transforms.ToTensor())
#mean_pre_trained, std_pre_trained = get_mean_devst(dataset_train)

#Database normalizzato per squeezenet preallenata
#print(mean_pre_trained)
#print(std_pre_trained)
mean_pre_trained = [0.47607468, 0.44641235, 0.39604222]
std_pre_trained = [0.27902635, 0.27250564, 0.27781429]

transformss = transforms.Compose([
                                      #transforms.Resize((width,height)),
                                      transforms.ToTensor(), 
                                      #transforms.Normalize(mean_pre_trained,std_pre_trained),
                                      transforms.Normalize(mean_squeezenet, std_squeezenet),
                                  ])
dataset_train_squeezenet = ScenesDataset('','train.txt',transform=transformss)
dataset_test_squeezenet = ScenesDataset('','test.txt',transform=transformss)

#Database normalizzato per regressione logistica.
transformss = transforms.Compose([
                                      #transforms.Resize((width,height)),
                                      transforms.ToTensor(), 
                                      #transforms.Normalize(mean_pre_trained,std_pre_trained),
                                      transforms.Normalize(mean_pre_trained, std_pre_trained),
                                  ])
dataset_train_logistic_regression = ScenesDataset('','train.txt',transform=transformss)
dataset_test_logistic_regression = ScenesDataset('','test.txt',transform=transformss)


'''
sample = dataset_train[0]
#l'immagine è 3 x 256 x 256 perché è una immagine a colori
print(sample)
print(sample['image'].shape)
print(sample['label'])
'''

# Creo dataset per calcolo accuracy su immagini mai viste.

In [ ]:
mean_pre_trained = [0.47607468, 0.44641235, 0.39604222]
std_pre_trained = [0.27902635, 0.27250564, 0.27781429]
transformss = transforms.Compose([
                                      #transforms.Resize((width,height)),
                                      transforms.ToTensor(), 
                                      #transforms.Normalize(mean_pre_trained,std_pre_trained),
                                      transforms.Normalize(mean_squeezenet, std_squeezenet),
                                  ])
dataset_mai_viste_squeezenet = ScenesDataset('','train_immagini_mai_viste.txt',transform=transformss)

#Database normalizzato per regressione logistica.
transformss = transforms.Compose([
                                      #transforms.Resize((width,height)),
                                      transforms.ToTensor(), 
                                      #transforms.Normalize(mean_pre_trained,std_pre_trained),
                                      transforms.Normalize(mean_pre_trained, std_pre_trained),
                                  ])
dataset_mai_viste_logistic_regression = ScenesDataset('','train_immagini_mai_viste.txt',transform=transformss)


# Creo l'oggetto DataLoader

In [ ]:
train_loader_squeezenet = DataLoader(dataset_train_squeezenet, batch_size=20, num_workers=0, shuffle=True)
test_loader_squeezenet = DataLoader(dataset_test_squeezenet, batch_size=20, num_workers=0)
train_loader_mai_viste_squeezenet = DataLoader(dataset_mai_viste_squeezenet, batch_size=20, num_workers=0)

train_loader_logistic_regression = DataLoader(dataset_train_logistic_regression, batch_size=20, num_workers=0, shuffle=True)
test_loader_logistic_regression = DataLoader(dataset_test_logistic_regression, batch_size=20, num_workers=0)
train_loader_mai_viste_logistic_regression = DataLoader(dataset_mai_viste_logistic_regression, batch_size=20, num_workers=0)

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs
    
    
class LogisticRegressor(nn.Module):
    def __init__(self, in_features, ou_features):
        """Costruisce un regressore logistico.
        Input:
        in_features: numero di feature in input (es. 30)"""
        super(LogisticRegressor, self).__init__() #richiamo il costruttore della superclasse
        #questo passo è necessario per abilitare alcuni meccanismi automatici dei moduli di PyTorch
        self.linear = nn.Linear(in_features, ou_features) #il regressore logistico restituisce probabilità
        #quindi il numero di feature di output è "1"
        self.logistic = nn.Sigmoid()
    def forward(self,x):
        """Definisce come processare l'input x"""
        logits = self.linear(x)
        if self.training: #se il modulo è in fase di training
        #la proprietà "training" è messa a disposizione dai meccanismi
        #automatici dei moduli di PyTorch
            return logits
        else: #se siamo in fase di test, calcoliamo le probabilità
            return self.logistic(logits)

# Variabili di controllo per l'allenamento:

In [ ]:
batch_size = 100
n_iters = 3000
epochs = n_iters / (len(dataset_train) / batch_size)
input_dim = width*height*3
output_dim = 2
lr_rate = 0.0001
channel = 3
momentum = 0.8

In [ ]:
model_logistic_regression = LogisticRegression(input_dim, output_dim)
model_squeezenet = models.squeezenet1_0(pretrained=True)#(input_dim, output_dim)
model_alexnet = models.alexnet(pretrained=True)#(input_dim, output_dim)
model_alexnet.classifier[6] = nn.Linear(4096, output_dim)

In [ ]:
def train_classification(model, train_loader, test_loader, lr=0.001, epochs=20, momentum=0.8, exp_name = 'experiment', logistic_regression = False):
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(),lr, momentum=momentum)
    loaders = {'train':train_loader, 'test':test_loader}
    losses = {'train':[], 'test':[]}
    accuracies = {'train':[], 'test':[]}
    
    
    loss_meter = AverageValueMeter()
    acc_meter = AverageValueMeter()
    loss_logger = VisdomPlotLogger('line', env=exp_name, opts={'title': 'Loss', 'legend':['train','test']})
    acc_logger = VisdomPlotLogger('line', env=exp_name, opts={'title': 'Accuracy','legend':['train','test']})
    visdom_saver = VisdomSaver(envs=[exp_name])

    if torch.cuda.is_available():
        model=model.cuda()
    
    for e in range(epochs):
        #print("Primo ciclo for.")
        for mode in ['train', 'test']:
            #print("Secondo ciclo for.")
            
            loss_meter.reset()
            acc_meter.reset()
            
            if mode=='train':
                model.train()
            else:
                model.eval()
            epoch_loss = 0
            epoch_acc = 0
            samples = 0
            #print("Mode-->",mode)
            for i, batch in enumerate(loaders[mode]):
                #trasformiamo i tensori in variabili
                if logistic_regression == True:
                    x=Variable(batch['image'].view(-1, width*height*channel), requires_grad=(mode=='train'))
                else:
                    x=Variable(batch['image'], requires_grad=(mode=='train'))

                y=Variable(batch['label'])
                if torch.cuda.is_available():
                    x, y = x.cuda(), y.cuda()
                    #print("Con cuda")
                #else:
                    #print("Senza cuda")
                output = model(x)
                #print(output)
                #print(y)
                l = criterion(output,y)
                if mode=='train':
                    l.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                #print("L-->",l.item())
                acc = accuracy_score(y.cpu().data,output.cpu().max(1)[1].data)
                epoch_loss+=l.data.item()*x.shape[0]
                epoch_acc+=acc*x.shape[0]
                samples+=x.shape[0]
                '''print ("\r[%s] Epoch %d/%d. Iteration %d/%d. Loss: %0.2f. Accuracy: %0.2f\t\t\t\t\t" % \
                (mode, e+1, epochs, i, len(loaders[mode]), epoch_loss/samples, epoch_acc/samples),
                epoch_loss/samples,
                epoch_acc/samples,
                losses[mode].append(epoch_loss))'''
                accuracies[mode].append(epoch_acc)
                n = batch['image'].shape[0]
                loss_meter.add(l.item()*n,n)
                acc_meter.add(acc*n,n)
                #loss_logger.log(e+(i+1)/len(loaders[mode]), loss_meter.value()[0], name=mode)
                #acc_logger.log(e+(i+1)/len(loaders[mode]), acc_meter.value()[0], name=mode)


            loss_logger.log(e+1, loss_meter.value()[0], name=mode)
            acc_logger.log(e+1, acc_meter.value()[0], name=mode)
            if mode == "train":
                print(e, end = " ")
            #print("Fine secondo ciclo for")
        '''print("\r[%s] Epoch %d/%d. Iteration %d/%d. Loss: %0.2f. Accuracy: %0.2f\t\t\t\t\t" % \
        (mode, e+1, epochs, i, len(loaders[mode]), epoch_loss, epoch_acc))'''
        torch.save(model.state_dict(), "./" + exp_name + ".pth")
        print("Pesi aggiornati.")
    #print("Ho finito.")
    #restituiamo il modello e i vari log
    return model, (losses, accuracies)

In [ ]:
lenet_mnist, lenet_mnist_logs = train_classification(model_logistic_regression, 
                                                     epochs = 50, 
                                                     train_loader = train_loader_logistic_regression,
                                                     test_loader = test_loader_logistic_regression, 
                                                     logistic_regression = True,
                                                     exp_name = "Train_logisticRegression_lr" + str(lr_rate) + "_momentum" + str(momentum) + "_epochs" + str(int(epochs)), 
                                                     lr = lr_rate, 
                                                     momentum = momentum)

In [ ]:
result_squeezenet, result_logs_squeezenet = train_classification(model_squeezenet, 
                                                     epochs = 50, 
                                                     train_loader = train_loader_squeezenet,
                                                     test_loader = test_loader_squeezenet, 
                                                     logistic_regression = False,
                                                     exp_name = "Train_squeezenet_lr" + str(lr_rate) + "_momentum" + str(momentum) + "_epochs" + str(int(epochs)), 
                                                     lr = lr_rate, 
                                                     momentum = momentum)

In [ ]:
result_alexnet, result_logs_alexnet = train_classification(model_alexnet, 
                                                     epochs = 50, 
                                                     train_loader = train_loader_squeezenet,
                                                     test_loader = test_loader_squeezenet, 
                                                     logistic_regression = False,
                                                     exp_name = "NuovaProva_Train_alexnet_lr" + str(lr_rate) + "_momentum" + str(momentum) + "_epochs" + str(int(epochs)), 
                                                     lr = lr_rate, 
                                                     momentum = momentum)

In [ ]:
def load_state(net, path):
    my_file = Path(path) #Carica il file(lo state potenzialmente)
    if my_file.is_file(): #Carico lo state altrimenti rialleno
        net.load_state_dict(torch.load(path))
        print("Caricato " + path)
    else: 
        print("File mancante.")

In [ ]:
load_state(model_alexnet, "./Train_alexnet_lr0.0001_momentum0.8_epochs53.pth")
load_state(model_squeezenet, "./Train_squeezenet_lr0.0001_momentum0.8_epochs53.pth")
load_state(model_logistic_regression, "./Train_logisticRegression_lr0.0001_momentum0.8_epochs53.pth")

In [ ]:
def test_model_classification(model, test_loader, logistic_regression = True):
    softmax = nn.Softmax(dim=1)
    model.eval()
    model.cpu()
    preds = []
    gts = []
    for batch in test_loader:
        #x=Variable(batch["image"])
        if logistic_regression == True:
            x=Variable(batch['image'].view(-1, width*height*channel))
        else:
            x=Variable(batch['image'])
        x = x.cpu()
        #applichiamo la funzione softmax per avere delle probabilità
        if torch.cuda.is_available():
            x = x.cuda()
            model.cuda()
        pred = softmax(model(x)).data.cpu().numpy().copy()
        gt = batch["label"].cpu().numpy().copy()
        #print("Pred-->", pred, ", gt-->", gt)
        preds.append(pred)
        gts.append(gt)
        #print(len(preds), len(gts))
    return np.concatenate(preds),np.concatenate(gts)

In [ ]:
def get_indicators(net, loader, train, logistic_regression):
    #In questo caso e' l'accuracy per la classe AlexNetRetrain
    predictions, groundruth_t = test_model_classification(net, loader, logistic_regression)
    if train == True:
        print("****Indicatori per dataset di Train****")
    else:
        print("****Indicatori per dataset di Test****")
    print ("Accuracy: %0.2f" % \
    accuracy_score(groundruth_t,predictions.argmax(1)))
    print("Precision_score-->")
    print(precision_score(groundruth_t,predictions.argmax(1), average = 'macro'))
    print("Recall_score-->")
    print(recall_score(groundruth_t,predictions.argmax(1), average = 'macro'))
    print("F1_Score-->")
    print(f1_score(groundruth_t,predictions.argmax(1), average = 'macro'))
    print("Matrice di confusione-->\n")
    print(confusion_matrix(groundruth_t,predictions.argmax(1)))


In [ ]:
print("------------------------------------------ Accuracy su immagini mai viste prima ------------------------------------------")
print("++++++++++ Logistic Regression ++++++++++")
get_indicators(model_logistic_regression, train_loader_mai_viste_logistic_regression, True, True)
print("++++++++++ Squeezenet ++++++++++")
get_indicators(model_squeezenet, train_loader_mai_viste_squeezenet, True, False)
print("++++++++++ AlexNet ++++++++++")
get_indicators(model_alexnet, train_loader_mai_viste_squeezenet, True, False)

In [ ]:
print("------------------------------------------ Accuracy su immagini di train ------------------------------------------")
print("++++++++++ Logistic Regression ++++++++++")
get_indicators(model_logistic_regression, train_loader_logistic_regression, True, True)
print("++++++++++ Squeezenet ++++++++++")
get_indicators(model_squeezenet, train_loader_squeezenet, True, False)
print("++++++++++ AlexNet ++++++++++")
get_indicators(model_alexnet, train_loader_squeezenet, True, False)

In [ ]:
print("------------------------------------------ Accuracy su immagini di test ------------------------------------------")
print("++++++++++ Logistic Regression ++++++++++")
get_indicators(model_logistic_regression, test_loader_logistic_regression, False, True)
print("++++++++++ Squeezenet ++++++++++")
get_indicators(model_squeezenet, test_loader_squeezenet, False, False)
print("++++++++++ AlexNet ++++++++++")
get_indicators(model_alexnet, test_loader_squeezenet, False, False)